In [18]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np


In [20]:
#@title
%%capture
%cd /content/drive/MyDrive/RohithWorkspace/Object-Detection-Website
!git config --global user.email "007rohitjj04@gmail.com"
!git config --global user.name "Rohith04MVK"
!git add -A
!git commit -m "Still testing google colab"
!git push

SyntaxError: ignored